# SLTimer Example Analysis of TDC2 Data

This notebook shows you how to find the estimation of a lens time delay from TDC2 light curve data using the `PyCS` code. For a detailed tutorial through the `PyCS` code, please visit this [address.](https://github.com/DarkEnergyScienceCollaboration/SLTimer/blob/master/notebooks/PyCS_Tutorial.ipynb)

First, we'll import `SLTimer`, as well as a few other important commands. 

In [2]:
from __future__ import print_function
%matplotlib inline

import sys
sys.path.append('../python')
import desc.sltimer

%load_ext autoreload
%autoreload 2

SyntaxError: invalid syntax (IO.py, line 204)

Next, let's simplify our variable. 

In [ ]:
timer = desc.sltimer.SLTimer()

Let's define our variables and download our trial data, testing it to make sure that we have downloaded the correct file. The output should state 194 lines if using the trialcurves.txt file. 

In [ ]:
url = "http://www.slac.stanford.edu/~pjm/LSST/DESC/SLTimeDelayChallenge/release/tdc2/gateway/tdc2-gateway-example.txt"
timer.download(url, and_read=True, format='tdc2')

In [ ]:
timer.display_light_curves(jdrange=(59500,63100))

These lines will run an entire free-knot spline technique on your data with a complete error analysis using the `TDC2` method. Below, you can specify how the time delays will be analyzed. The default is listed below according to the `PyCS` tutorial. See the bottom of the page for alternate methods. 

In [ ]:
timer.estimate_time_delays(method='pycs',microlensing='spline',agn='spline',error=None)

In [ ]:
timer.report_time_delays()

In [ ]:
timer.display_light_curves(jdrange=(59500,63100))

In [ ]:
timer.estimate_uncertainties(n=1,npkl=4)